# Practica Final

Aldao Amoedo, Héctor

Cabaleiro Pintos, Laura

Cotardo Valcárcel, Donato José

Romero Conde, José

---

## Preparación de los datos

El punto 1, está completado, aunque creo que el 1.3, donde preparamos los datos transformandolos y rellanando los nulos está mal.

### 1. Cargar los datos y descripción

In [1]:
using CSV
using DataFrames

df = CSV.read("Datos_Practica_Evaluacion_1.csv", DataFrame)

num_instancias, num_variables = size(df)
num_individuos = length(unique(df[:, 1]))
num_clases_salida = length(unique(df[:, end]))

println("Número de variables: $num_variables")
println("Número de instancias: $num_instancias")
println("Número de individuos: $num_individuos")
println("Número de clases de salida: $num_clases_salida")

Número de variables: 563
Número de instancias: 10299
Número de individuos: 30
Número de clases de salida: 6


Los datos dados ya están cargados, y como podemos observar tienen:
* 563 Variables
* 10299 Instancias
* 30 Individuos
* 6 Clases de salida

### 2. Calcular porcentaje de nulos

In [2]:
println("Porcentaje de nulos por variable:")
num_nulos_totales = 0
for col ∈ names(df)
    num_nulos = count(ismissing, df[:, col])
    num_nulos_totales += num_nulos
    porcentaje_nulos = (num_nulos / num_instancias) * 100
    println("$col: $porcentaje_nulos% nulos")
end

porcentaje_nulos_totales = (num_nulos_totales / (num_instancias * num_variables)) * 100
println("Porcentaje total de nulos en el conjunto: $porcentaje_nulos_totales%")

Porcentaje de nulos por variable:
subject: 0.0% nulos
tBodyAcc-mean()-X: 0.0% nulos
tBodyAcc-mean()-Y: 0.0% nulos
tBodyAcc-mean()-Z: 0.0% nulos
tBodyAcc-std()-X: 0.01941936110301971% nulos
tBodyAcc-std()-Y: 0.0% nulos
tBodyAcc-std()-Z: 0.01941936110301971% nulos
tBodyAcc-mad()-X: 0.01941936110301971% nulos
tBodyAcc-mad()-Y: 0.029129041654529564% nulos
tBodyAcc-mad()-Z: 0.029129041654529564% nulos
tBodyAcc-max()-X: 0.01941936110301971% nulos
tBodyAcc-max()-Y: 0.0% nulos
tBodyAcc-max()-Z: 0.009709680551509855% nulos
tBodyAcc-min()-X: 0.0% nulos
tBodyAcc-min()-Y: 0.0% nulos
tBodyAcc-min()-Z: 0.0% nulos
tBodyAcc-sma(): 0.009709680551509855% nulos
tBodyAcc-energy()-X: 0.0% nulos
tBodyAcc-energy()-Y: 0.0% nulos
tBodyAcc-energy()-Z: 0.0% nulos
tBodyAcc-iqr()-X: 0.029129041654529564% nulos
tBodyAcc-iqr()-Y: 0.009709680551509855% nulos
tBodyAcc-iqr()-Z: 0.0% nulos
tBodyAcc-entropy()-X: 0.0% nulos
tBodyAcc-entropy()-Y: 0.0% nulos
tBodyAcc-entropy()-Z: 0.0% nulos
tBodyAcc-arCoeff()-X,1: 0.0% nulo

### 3. Preparar los datos para las técnicas de clasificación

Para rellenar valores faltantes tenemos que hacernos una idea de que tipo de datos encontraremos.

In [3]:
Set([eltype(df[!,col]) for col in names(df)])

Set{Type} with 4 elements:
  Float64
  Union{Missing, Float64}
  String31
  Int64

In [4]:
sum([eltype(df[!,col])==String31 for col in names(df)]) # la ultima

1

In [5]:
sum([eltype(df[!,col])==Int64 for col in names(df)]) # la primera

1

Es decir, todas las columnas que tienen valores faltantes son de tipo númerico

In [6]:
using Statistics
using StatsBase

for col ∈ names(df)
    if eltype(df[!, col]) == Union{Missing, Float64}
        media_columna = mean(skipmissing(df[!, col]))
        cuantos_nulos = sum(ismissing.(df[!, col]))
        println("Columna $col reemplaza a sus $cuantos_nulos nulos por $media_columna")
        df[ismissing.(df[!, col]), col] .= media_columna
        df[!, col] = Float64.(df[!, col]) 
    end
end

println("Valores nulos rellenados.")

Columna tBodyAcc-std()-X reemplaza a sus 2 nulos por -0.607824116906722
Columna tBodyAcc-std()-Z reemplaza a sus 2 nulos por -0.6129910690837637
Columna tBodyAcc-mad()-X reemplaza a sus 2 nulos por -0.6336147148236936
Columna tBodyAcc-mad()-Y reemplaza a sus 3 nulos por -0.5256922189005827
Columna tBodyAcc-mad()-Z reemplaza a sus 3 nulos por -0.6151836920835089
Columna tBodyAcc-max()-X reemplaza a sus 2 nulos por -0.46672533928067816
Columna tBodyAcc-max()-Z reemplaza a sus 1 nulos por -0.5622065339628413
Columna tBodyAcc-sma() reemplaza a sus 1 nulos por -0.5520443167787891
Columna tBodyAcc-iqr()-X reemplaza a sus 3 nulos por -0.6892991087281277
Columna tBodyAcc-iqr()-Y reemplaza a sus 1 nulos por -0.6435385345032292
Columna tBodyAccJerk-std()-X reemplaza a sus 1 nulos por -0.6398212084978457
Columna tBodyAccJerk-mad()-X reemplaza a sus 4 nulos por -0.6368691570541302
Columna tBodyAccJerk-mad()-Y reemplaza a sus 4 nulos por -0.5940209963793627
Columna tBodyAccJerk-mad()-Z reemplaza a 

Convertimos variables categóricas a numéricas usando One-Hot Encoding

In [7]:
clases = String[]#se usará más tarde
for col in names(df)
    if eltype(df[!, col]) == String31
        categorias = unique(df[!, col])
        for categoria in categorias
            nombre = "$(col)_$(categoria)"
            push!(clases, nombre)
            nueva_columna = Symbol(nombre)
            df[!, nueva_columna] = df[!, col] .== categoria
        end
        select!(df, Not(col)) # eliminamos la columna original
    end
end

println("One-Hot Encoding aplicado exitosamente.")

One-Hot Encoding aplicado exitosamente.



Validamos los datos

In [8]:
println("Validación de datos:")

# Calcular la cantidad total de valores nulos
total_nulos = sum(col -> count(ismissing, col), eachcol(df))
println("Valores nulos restantes: $total_nulos")

# Verificar tipo de datos de cada columna
println("Tipos de datos por columna:")
for col in names(df)
    println("$col: $(eltype(df[!, col]))")
end

Validación de datos:
Valores nulos restantes: 0
Tipos de datos por columna:
subject: Int64
tBodyAcc-mean()-X: Float64
tBodyAcc-mean()-Y: Float64
tBodyAcc-mean()-Z: Float64
tBodyAcc-std()-X: Float64
tBodyAcc-std()-Y: Float64
tBodyAcc-std()-Z: Float64
tBodyAcc-mad()-X: Float64
tBodyAcc-mad()-Y: Float64
tBodyAcc-mad()-Z: Float64
tBodyAcc-max()-X: Float64
tBodyAcc-max()-Y: Float64
tBodyAcc-max()-Z: Float64
tBodyAcc-min()-X: Float64
tBodyAcc-min()-Y: Float64
tBodyAcc-min()-Z: Float64
tBodyAcc-sma(): Float64
tBodyAcc-energy()-X: Float64
tBodyAcc-energy()-Y: Float64
tBodyAcc-energy()-Z: Float64
tBodyAcc-iqr()-X: Float64
tBodyAcc-iqr()-Y: Float64
tBodyAcc-iqr()-Z: Float64
tBodyAcc-entropy()-X: Float64
tBodyAcc-entropy()-Y: Float64
tBodyAcc-entropy()-Z: Float64
tBodyAcc-arCoeff()-X,1: Float64
tBodyAcc-arCoeff()-X,2: Float64
tBodyAcc-arCoeff()-X,3: Float64
tBodyAcc-arCoeff()-X,4: Float64
tBodyAcc-arCoeff()-Y,1: Float64
tBodyAcc-arCoeff()-Y,2: Float64
tBodyAcc-arCoeff()-Y,3: Float64
tBodyAcc-arCo

In [9]:
CSV.write("DatosPreprocesados.csv",df) # por si mas tarde se quieren importar directamente

"DatosPreprocesados.csv"

In [10]:
# df = CSV.read("DatosPreprocesados.csv", DataFrame)

### 4. Segmentar el 10% de los datos usando HoldOut

In [11]:
using Random
Random.seed!(172)

individuos = unique(df[:, :subject])  # Extraer individuos

num_holdout = Int(round(0.1 * length(individuos)))  # 10% de los individuos
holdout_individuos = shuffle(individuos)[1:num_holdout]  # Seleccionar individuos aleatorios

holdout_df = filter(fila -> fila.subject in holdout_individuos, df)
train_df = filter(fila -> !(fila.subject in holdout_individuos), df)

println("Individuos en el holdout: ", holdout_individuos)
println("Tamaño del conjunto de entrenamiento: $(size(train_df)[1])")
println("Tamaño del conjunto de holdout: $(size(holdout_df)[1])")


Individuos en el holdout: [4, 12, 29]
Tamaño del conjunto de entrenamiento: 9318
Tamaño del conjunto de holdout: 981


In [12]:
holdout_subjects = unique(holdout_df[:, :subject])
train_subjects = unique(train_df[:, :subject])

intersection = intersect(holdout_subjects, train_subjects)

if isempty(intersection)
    println("La división es correcta: no hay individuos compartidos entre los conjuntos.")
else
    println("Error: Hay individuos compartidos entre los conjuntos.")
    println("Individuos compartidos: $intersection")
end

La división es correcta: no hay individuos compartidos entre los conjuntos.


### 5. Hacer 5 fold cross-validation

In [13]:
n_folds = 5

fold_indices = randperm(length(individuos))  # Permutar aleatoriamente los índices
fold_tamano = Int(round(length(individuos) / n_folds))
folds = [individuos[fold_indices[(i - 1) * fold_tamano + 1:min(i * fold_tamano, end)]] for i in 1:n_folds]

# Crear folds a nivel de instancias
fold_data = []
for fold in folds
    push!(fold_data, filter(fila -> fila.subject in fold, df)[!,2:end]) # quitamos la columna subject que ya no nos importa
    #print(filter(fila -> fila.subject in fold, df)[1,1])
end

# Imprimir resumen de los folds
for (i, fold) in enumerate(fold_data)
    println("Fold $i:")
#    println("Número de participantes: $(length(unique(fold[:, :subject])))")
    println("Número de instancias: $(size(fold, 1))")
 #   println("Participantes: $(unique(fold[:, :subject]))")
end


Fold 1:
Número de instancias: 1983
Fold 2:
Número de instancias: 2167
Fold 3:
Número de instancias: 1962
Fold 4:
Número de instancias: 1977
Fold 5:
Número de instancias: 2210


### 6. Normalizar usando MinMaxScaler

In [14]:
function MinMaxScaler(columna)
    dt = fit(UnitRangeTransform, (columna))
    return StatsBase.transform(dt, columna)
end

for i ∈ 1:length(fold_data)
    for col in names(fold_data[i])
        if eltype(fold_data[i][:,col]) == Float64
            fold_data[i][:,col] = MinMaxScaler(fold_data[i][:,col])
        end
    end
    println("Normalizado el fold $i")
end

Normalizado el fold 1
Normalizado el fold 2
Normalizado el fold 3
Normalizado el fold 4
Normalizado el fold 5


---

## Creación de los modelos básicos

### 7. Filtrado

#### 7.1 ANOVA

In [15]:
using Distributions

function anova(fold,α) # diapositiva 26
    
    # División de Datos en Grupos
    
    grupos = DataFrame[] 
    for clase in clases
        push!(grupos,filter(fila -> fila[clase] == 1,fold))
    end
    println(typeof(grupos[1]))
    
    # Cálculo de la Variabilidad
    
    medias = Array{Float64}(undef, 0, size(grupos[1],2)) 
    for grupo in grupos
        media_grupo = Float64[]
        for col in names(grupo)        
            push!(media_grupo, mean(grupo[!,col]))
        end
        medias = vcat(medias, transpose(media_grupo))
    end 
    println(size(medias))
    medias_entre_grupos = [mean(medias[:][i]) for i in 1:size(medias,2)]
    
    # usaremos implicitamente el hecho de que los grupos tienen el mismo numero de elementos
    # notación de de https://en.wikipedia.org/wiki/One-way_analysis_of_variance
    
    Sb = [sum((medias[:,i] .-  medias_entre_grupos[i]).^2) for i in 1:size(medias,2)]

    Sw = zeros(size(medias,2))
    for (i, grupo) in enumerate(grupos) 
        for (j, col) in enumerate(names(grupo))
            Sw[j] += sum((grupo[!,col] .- medias[i,j]).^2)
        end
    end
    
    # Cálculo del Estadístico F
    
    F = (Sb./(length(clases)-1)) ./ (Sw./(size(fold,1)-length(clases)))
    
    # Determinación de la Significancia
    
    distribucionF = FDist(length(clases)-1, size(fold,1)-length(clases))
    indices = BitVector(F .< quantile.(distribucionF,α))
    indices[end-6:end].=1
    return indices
end

for i in 1:length(fold_data)
    indices = anova(fold_data[i],0.95)
    print(indices) # vemos con un cero las variables que se dejan fuera
    fold_data[i] = fold_data[i][!,indices]
end

DataFrame
(6, 567)
Bool[1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [16]:
fold_data # vemos que ahora hay menos columnas

5-element Vector{Any}:
 1983×117 DataFrame
  Row │ tBodyAcc-mean()-X  tBodyAcc-mean()-Z  tBodyAcc-arCoeff()-X,3  tBodyAcc ⋯
      │ Float64            Float64            Float64                 Float64  ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │          0.649526           0.608771                0.411564           ⋯
    2 │          0.641366           0.605877                0.501627
    3 │          0.597865           0.624954                0.47563
    4 │          0.65034            0.632561                0.54715
    5 │          0.648355           0.552065                0.509295           ⋯
    6 │          0.637153           0.619647                0.560254
    7 │          0.635201           0.629293                0.497573
    8 │          0.643243           0.601442                0.503591
    9 │          0.640751           0.61769                 0.561298           ⋯
   10 │          0.763661           0.301878               

#### 7.2 Mutual Information

In [17]:
function kargmax(v, n) 
    indices_menor_a_mayor = partialsortperm(v, 1:length(v))
    #print(indices_menor_a_mayor)
    return sort(last(indices_menor_a_mayor,n))
end
x = [3,6,2,7,4,5,1,4]
kargmax(x, 4)

4-element Vector{Int64}:
 2
 4
 6
 8

In [20]:
 using InformationMeasures

for i ∈ 1:length(fold_data) # para cada fold
    explicativas = []
    for x ∈ names(fold_data[i]) # por cada variable explicativa
        if eltype(fold_data[i][!,x]) == Float64
             # por cada variable respuesta
            ix = mean([get_mutual_information(fold_data[i][!,x], fold_data[i][!,y]) for y ∈ last(names(fold_data[i]),6)])
            push!(explicativas, ix)
            #println("Fold $i, variable $x, informacion = $ix")
        end
    end
    #println(explicativas)
    indices = kargmax(explicativas,58) # cogemos las 64 variables más explicativas
    for respuesta ∈ length(names(fold_data[i]))-6:length(names(fold_data[i]))
        if respuesta ∉ indices
            push!(indices, respuesta)
        end
    end
    fold_data[i] = fold_data[i][!,indices]
end


In [19]:
fold_data # vemos que ahora hay 64 columnas

5-element Vector{Any}:
 1983×117 DataFrame
  Row │ tBodyAcc-mean()-X  tBodyAcc-mean()-Z  tBodyAcc-arCoeff()-X,3  tBodyAcc ⋯
      │ Float64            Float64            Float64                 Float64  ⋯
──────┼─────────────────────────────────────────────────────────────────────────
    1 │          0.649526           0.608771                0.411564           ⋯
    2 │          0.641366           0.605877                0.501627
    3 │          0.597865           0.624954                0.47563
    4 │          0.65034            0.632561                0.54715
    5 │          0.648355           0.552065                0.509295           ⋯
    6 │          0.637153           0.619647                0.560254
    7 │          0.635201           0.629293                0.497573
    8 │          0.643243           0.601442                0.503591
    9 │          0.640751           0.61769                 0.561298           ⋯
   10 │          0.763661           0.301878               

#### 7.3 RFE con el método de LogisticRegression con una eliminación del 50% de las variables en cada pasada.

In [63]:
using GLM

for i ∈ 1:length(fold_data) # para cada fold
    #fm = @formula(fold_data[i][!,Activity_WALKING] ~  fold_data[i][!,tBodyAcc-mean()-X] )
    #logit = glm(fm, train, Binomial(), ProbitLink())
    explicativas = []
    respuesta = []
    for col in names(fold_data[i])
        if eltype(fold_data[i][!,col]) == Float64
            push!(explicativas,col)
        elseif eltype(fold_data[i][!,col]) == Bool
            push!(respuesta,col)
        end
    end
 
    y = Term(Symbol(respuesta[1]))
    x =  +((Term(Symbol(i)) for i ∈ explicativas)...) # https://discourse.julialang.org/t/non-call-expression-encountered/90725/2
    formula = y ~  x
    logistica = glm(formula, fold_data[i], Binomial(), ProbitLink())
    print(logistica)
end


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, ProbitLink}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

Activity_STANDING ~ 1 + tBodyAcc-mean()-X + tBodyAcc-mean()-Z + tBodyAcc-arCoeff()-Y,1 + tBodyAcc-arCoeff()-Z,1 + tBodyAcc-arCoeff()-Z,2 + tBodyAcc-correlation()-Y,Z + tGravityAcc-std()-X + tGravityAcc-std()-Y + tGravityAcc-mad()-Z + tGravityAcc-max()-Z + tGravityAcc-min()-Y + tGravityAcc-iqr()-Y + tGravityAcc-iqr()-Z + tGravityAcc-entropy()-X + tGravityAcc-arCoeff()-X,1 + tGravityAcc-arCoeff()-Y,1 + tGravityAcc-arCoeff()-Y,3 + tGravityAcc-arCoeff()-Z,4 + tBodyAccJerk-mean()-Y + tBodyAccJerk-mean()-Z + tBodyAccJerk-arCoeff()-Z,1 + tBodyGyro-entropy()-Z + tBodyGyro-arCoeff()-X,1 + tBodyGyro-arCoeff()-Z,1 + tBodyGyro-arCoeff()-Z,3 + tBodyGyro-correlation()-Y,Z + tBodyGyroJerk-mean()-Z + tBodyGyroJerk-arCoeff()-X,1 + tBodyGyroJerk-arCoeff()-Z,1 + tBodyGyroJ

### 8. Reducción dimensionalidad

#### 8.1 PCA

#### 8.2 LDA

#### 8.3 ICA

#### 8.4 Isomap

#### 8.5 LLE

### 9. Clasificadores

#### 9.1 MLP con al menos las siguientes arquitecturas: [50], [100] [100, 50]

#### 9.2 KNN con valores de vecindario entre 1, 10 y 20

#### 9.3 SVM con el parámetro C con valores 0.1, 0.5 y 1.0

---

## Creación de los modelos ensemble

---

## Conclusiones